# Hello MLflow!

In this notebook, we'll say hello with MLflow.

In [ ]:
import warnings
from pathlib import Path

import mlflow
import numpy as np
from mlflow.tracking import MlflowClient

warnings.filterwarnings("ignore")

In [ ]:
print("MLflow version:", mlflow.__version__)

## Starting Experiment

In [ ]:
EXPERIMENT_NAME = "first-experiment"

In [ ]:
# set experiment name
mlflow.set_experiment(EXPERIMENT_NAME)

After running the above cell, there should be a folder named `mlruns` in the same directory with this notebook.

In [ ]:
mlflow.get_tracking_uri()

In [ ]:
mlflow.get_registry_uri()

We will use the function below to run (dummy) experiment using MLflow.

In [ ]:
def start_experiment(experiment_name, run_name="", output_dir="outputs"):
    rng = np.random.default_rng()
    # set experiment name
    mlflow.set_experiment(
        experiment_name=experiment_name
    )

    # use context manager to start runs
    with mlflow.start_run(run_name=run_name) as run:
        print("Running experiment..")
        print("Run:", run.info.run_uuid)

        # set some tags
        mlflow.set_tag("sample_name", run_name)
        mlflow.set_tag("run_file", "jupyter_notebook")

        # log some parameters
        mlflow.log_param("sample_param", rng.integers(0, 100))

        # log some metrics
        mlflow.log_metric("accuracy", rng.random())
        mlflow.log_metric("f1_score", rng.random())

        # to log artifacts, we need to have them in directory first
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir(parents=True)
        model_filename = output_dir / "model.joblib"
        with model_filename.open("w") as f:
            f.write("This is a model!")

        mlflow.log_artifact(str(output_dir), artifact_path="artifact")

In [ ]:
start_experiment(EXPERIMENT_NAME, run_name="run-1")

In [ ]:
start_experiment(EXPERIMENT_NAME, run_name="run-2")

## Inspecting Experiments and Runs

After running some experiments, we can inspect them using different ways. MLflow provide several approach to do this:
* using MLflow Tracking UI
* querying programmatically

### Managing Experiments and Runs Programmatically

To organize experiments and runs in it, typically, we use `mlflow.client` module or simply `MlflowClient`.

In [ ]:
client = MlflowClient()

In [ ]:
experiment = client.get_experiment_by_name(EXPERIMENT_NAME)
experiment

In [ ]:
experiment.experiment_id

In [ ]:
runs = client.search_runs(
    experiment_ids=[experiment.experiment_id]
)

In [ ]:
len(runs)

In [ ]:
runs[0]

In [ ]:
runs[0].to_dictionary()

### Managing Experiments and Runs with Tracking UI

If we store the metadata locally (inside `mlruns/`), then we could directly run `mlflow ui` command in the terminal to open up the tracking UI. The steps are:
1. make sure you're already in the desired folder where there's `mlruns` folder
2. launch terminal and run command: `mlflow ui`
3. open browser and go to `localhost:5000`

The default port is `5000`, but you can always change it using option `-p PORT`.

![](../assets/img/mlflow_ui.png)